# Learner Utils

> utils used by the learner class


In [ ]:
#| default_exp learner_utils

In [ ]:
#| hide
from nbdev.showdoc import *  # type: ignore # noqa: F403

In [ ]:
#| export
import os
import random
from copy import deepcopy
import numpy as np
import torch.nn as nn
import torch
from fedai.vision.VisionBlock import VisionBlock
from fedai.vision.models import *
from fedai.text.models import *
from fedai.models import * # noqa: F403
from peft import *  # type: ignore # noqa: F403


In [ ]:
#| export
import importlib
def get_cls(module_name, class_name):
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

In [ ]:
#| export
def get_block(cfg, id, train=True):
    block = VisionBlock if cfg.data.modality == ['Vision'] else None
    return block(cfg, id, train=train)

In [ ]:
#| export
def get_model(cfg):
    model_name = cfg.model.name

    # Set seed before model creation to ensure the same initialization (cleints start from the same model)
    torch.manual_seed(cfg.random_seed)  
    np.random.seed(cfg.random_seed)
    random.seed(cfg.random_seed)

    # Check if the model name contains "hf://"
    if model_name.startswith("hf://"):
        return get_hf_model(cfg)  
    
    mapping = {

                "LogisticRegression": LogisticRegression(  
                    input_dim=getattr(cfg.model, "input_dim", 784),  
                    num_classes=getattr(cfg.model, "num_classes", 10)
                ),

                "MLP": MLP(  
                    input_dim=getattr(cfg.model, "input_dim", 784),  
                    hidden_dim=getattr(cfg.model, "hidden_dim", 128),  
                    num_classes=getattr(cfg.model, "num_classes", 10)
                ),

                "MNISTCNN": MNISTCNN(
                    in_channels=getattr(cfg.model, "in_channels", 1),
                    img_size=getattr(cfg.model, "img_size", 28),  
                    hidden_dim=getattr(cfg.model, "hidden_dim", 256),  
                    num_classes=getattr(cfg.model, "num_classes", 10)
                ),  

                "CIFAR10Model": CIFAR10Model(
                    in_channels=getattr(cfg.model, "in_channels", 3),
                    img_size=getattr(cfg.model, "img_size", 32),  
                    hidden_dim=getattr(cfg.model, "hidden_dim", 512),  
                    num_classes=getattr(cfg.model, "num_classes", 10)
                ),

                "CharacterLSTM": CharacterLSTM(  
                    vocab_size=getattr(cfg.model, "vocab_size", 50000),  
                    embed_size=getattr(cfg.model, "embed_size", 512),  
                    hidden_size=getattr(cfg.model, "hidden_dim", 512),  
                    num_layers=getattr(cfg.model, "num_layers", 8)
                )

            }


    # Look up the model in the mapping
    if model_name in mapping:
        return mapping[model_name]
    
    raise ValueError(f"Model '{model_name}' is not recognized, the available models are: {list(mapping.keys())}")


In [ ]:
#| export
def get_criterion(customm_fn):
    if customm_fn:
        return customm_fn
    else:
        return nn.CrossEntropyLoss()

in the case of one model aggregation, we send the aggregated model back to all clients. On the other hand, Personalized FL, FMTL, ...etc uses one model per client so we need to only work on per client model case.

In [ ]:
#| export
def load_state_from_disk(cfg, state, latest_round, id, t, state_dir):


    if cfg.agg == "one_model":
        global_model_path = os.path.join(cfg.save_dir, str(t-1), "global_model", "state.pth")
        gloabal_model_state = torch.load(global_model_path, weights_only= False)

        if isinstance(state["model"], torch.nn.Module) or isinstance(state["model"], dict) :
            state["model"].load_state_dict(gloabal_model_state["model"])
            print(f"Loaded Global model state from {global_model_path}")

        else:
            set_peft_model_state_dict(state["model"], gloabal_model_state["model"], "default")  # type: ignore

    else:

        if id not in latest_round:
            if cfg.client_cls == "pFedMe":
                state["persionalized_model_bar"] = deepcopy(state['model'])
            return state
        
        latest_comm_round = latest_round[id]
        
        old_state_path = os.path.join(cfg.save_dir, str(latest_comm_round), f"{state_dir}{id}", "state.pth")
        old_saved_state = torch.load(old_state_path, weights_only= False)

        if isinstance(state["model"], nn.Module) or isinstance(state["model"], dict) :
            state["model"].load_state_dict(old_saved_state["model"])

            if cfg.client_cls == "DMTL":
                state["h_c"] = old_saved_state["h_c"]
                state["h"] = state["h_c"]

            if cfg.client_cls == "pFedMe":
                state["persionalized_model_bar"] = old_saved_state["persionalized_model_bar"]
            
            print(f"Loaded client model state from {old_state_path}")
        else:
            set_peft_model_state_dict(state["model"], old_saved_state["model"], "default") # type: ignore

    return state

In [ ]:
#| hide
import nbdev # type: ignore
nbdev.nbdev_export() # type: ignore  # noqa: E702
